In [1]:
# df_players
# name, elo, state (H/m), move time avg (not possible yet), acpl game avg
# df_games
# p1_ID, p2_ID, winner_ID, date, opening, p1_ELO, p2_ELO
# df_moves
# player_ID, game_ID, time, eval, fen/move, comment

In [2]:
import pandas as pd
import numpy as np
import uuid        # for id generation

import chess.pgn
from stockfish import Stockfish

In [3]:
# stockfish = Stockfish(
#     'stockfish_14_linux_x64/stockfish_14_linux_x64/stockfish_14_x64', 
#     parameters={"Threads": 2, 'Min Split Depth': 26, 'Ponder':True}
# )
# stockfish.set_elo_rating(2600)
# stockfish.set_skill_level(30)

In [4]:
%%time

players = {
    'White':[],
    'White_Elo': [],
    'Black': [],
    'Black_Elo': [],
    'WhiteIsComp':[],
}

games = {
    'Game_ID': [],
    'Date' : [],
    'White':[],  # Dummy ID
    'White_Elo': [],
    'Black': [],  # Dummy ID
    'Black_Elo': [],
    'ECO': [],
    'Result': [],
}

moves_log_dict = {
    'Game_ID': [],
    'White': [],  # Dummy ID
    'Black': [],  # Dummy ID
    'ECO': [],
    'FEN_moves': [],
    #'cpl': [],
    'WhiteIsComp': [],
    'Result': [],
}

# read file
pgn = open("data/Fics_data_pc_data.pgn", encoding='UTF-8')  # always a Comp vs Player
game_counter = 0

while True:  # keep reading games
    try:
        game = chess.pgn.read_game(pgn)
        board = game.board()
        moves = list(game.mainline_moves())
        
        # Player
        players['White_Elo'].append(game.headers['WhiteElo'])
        players['Black_Elo'].append(game.headers['BlackElo'])
        players['White'].append(game.headers['White'])
        players['Black'].append(game.headers['Black'])
        players['WhiteIsComp'].append(game.headers.get('WhiteIsComp', 'No'))
        
        # Games
        games['Game_ID'].append(game.headers['FICSGamesDBGameNo'])
        games['White'].append(game.headers['White'])  # dummy ID
        games['Black'].append(game.headers['Black'])  # dummy ID
        games['White_Elo'].append(game.headers['WhiteElo'])
        games['Black_Elo'].append(game.headers['BlackElo'])
        games['ECO'].append(game.headers['ECO'])
        games['Result'].append(game.headers['Result'])
        games['Date'].append(game.headers['Date'])
        
        # Moves
        fen_pos = []
        acpl = []
        
        moves_log_dict['Game_ID'].append(game.headers['FICSGamesDBGameNo'])  # dummy ID for this dataset only breaks
        moves_log_dict['White'].append(game.headers['White'])  # dummy ID
        moves_log_dict['Black'].append(game.headers['Black'])  # dummy ID
        moves_log_dict['ECO'].append(game.headers['ECO'])
        moves_log_dict['WhiteIsComp'].append(game.headers.get('WhiteIsComp', 'No'))
        moves_log_dict['Result'].append(game.headers['Result'])

        # MOVE CYCLE
        for move in moves:
            board.push(move)
            fen_pos.append(board.fen())
            #stockfish.set_fen_position(board.fen())  # load stockfish with current FEN for eval
#             cpl = stockfish.get_evaluation()['value']/100
#             acpl.append(cpl)
        
        moves_log_dict['FEN_moves'].append(fen_pos)
        #moves_log_dict['cpl'].append(acpl)
        
        game_counter += 1
        if game_counter == 55:  # number of games to read
            break
    except AttributeError:  # no further games to read
        print('No further games to load.')
        break

print(f'{game_counter} games read.')
#TODO takes ~1 sec to process 5 games, too slow.

55 games read.
CPU times: user 496 ms, sys: 0 ns, total: 496 ms
Wall time: 494 ms


In [5]:
#generates unique IDs
def id_generator(id):
    return uuid.uuid4().int

## df_players

In [6]:
df_players = pd.DataFrame(players)
df_players.head()

,White,White_Elo,Black,Black_Elo,WhiteIsComp
0,forlat,1970,Geforce,2204,Yes
1,Geforce,2201,forlat,1973,No
2,forlat,1976,Geforce,2198,Yes
3,Geforce,2211,forlat,1963,No
4,forlat,1958,Geforce,2216,Yes


In [7]:
#pgn = open("data/Fics_data_pc_data.pgn", encoding='UTF-8') 

In [8]:
#game = chess.pgn.read_game(pgn)

In [9]:
# def players_id_list(input_df, c_black, c_white): # c_white, c_black - columns of respective df, must be a string
#     black = input_df[c_black]
#     white = input_df[c_white]
#     players_names = list(set(list(black) + list(white)))
#     df_pl = {'Players': players_names}
#     players_id = pd.DataFrame(df_pl)
#     players_id['Player_ID'] = players_id['Players'].apply(id_generator)
#     return players_id

#download list? input_df.to_csv("/notebooks/data/players_id")

In [10]:
players_id = pd.DataFrame({'Players': [], 'Player_ID' : []})

def players_id_list(input_df, players_id):
    #extract black and white columns
    black = list(input_df["Black"]) 
    white = list(input_df["White"])
    
    #merge uniqe values from both columns:
    bw_merged = pd.DataFrame(list(set(black + white)), columns=["Players"])
    
    # Player_ID filled with NaNs:
    players_id = players_id.merge(bw_merged, how="outer", left_on=["Players"], right_on=["Players"])
    
    # NaNs replaced with generated IDs
    nans_to_ids = players_id["Player_ID"].fillna(players_id["Player_ID"].apply(id_generator))
    
    #inserting missing IDs to players_id
    players_id["Player_ID"] = nans_to_ids 
    return players_id

In [11]:
players_id = players_id_list(df_players, players_id)

In [12]:
players_id

,Player_ID,Players
0,209652369178910039682878605522698113281,nakshatra
1,162128005018550880120099287906501078633,IFDThor
2,295534627959353627244905991660722810287,sparse
3,222267710200361995835915597876504557392,forlat
4,236783815257576772374598019559686508737,chesspickle
5,279906117560896678476901953433729609141,sussexman
6,16291973528981053809950622758054488371,Ghannoum
7,102244471177817299485982229777807084361,Bjboy
8,264836249963088507674927291041203925788,Arsyah
9,21193755420477066191904520857406789255,RamMiguel


In [13]:
df_dummy = pd.DataFrame({'White' : ["12345", 'DummyName', "1234", "forlat", "Geforce"], "Black" : ['DummyName', "12345", "Dummy", "Geforce", "Bambi"]})

In [30]:
df_dummy

,White,Black
0,12345,DummyName
1,DummyName,12345
2,1234,Dummy
3,forlat,Geforce
4,Geforce,Bambi


In [27]:
players_id  = players_id_list(df_dummy, players_id)

In [28]:
def assign_player_id(input_df): # returns a df with 2 new columns and assigned player ID
    #print("List od IDs has been generated:")
    #print(players_unique)
    m_white = input_df.merge(players_id, left_on=["White"], right_on=['Players'])   #
    m_white['White_ID'] = m_white['Player_ID']
    m_white.drop(columns=['Players', "Player_ID"], inplace=True)
    m_bw = m_white.merge(players_id, left_on=["Black"], right_on=['Players'])
    m_bw['Black_ID'] = m_bw['Player_ID']
    m_bw.drop(columns=['Players', "Player_ID"], inplace=True)
    df_players = m_bw
    return df_players

In [29]:
assign_player_id(df_dummy)

,White,Black,White_ID,Black_ID
0,12345,DummyName,95495513823953837683688792245195708558,307523617727438015006020371002052590422
1,DummyName,12345,307523617727438015006020371002052590422,95495513823953837683688792245195708558
2,1234,Dummy,189046562648961511944371207568674273504,144212631459099877414502536425082469363
3,forlat,Geforce,222267710200361995835915597876504557392,158946205730077771476317864809691404687
4,Geforce,Bambi,158946205730077771476317864809691404687,9812490742598205871836738644611487616


In [16]:
# Fix it:
def update_and_assign_player_IDs(input_df, players_id):
    df_players = assign_player_id(input_df)
    return df_players

In [17]:
update_and_assign_player_IDs(df_dummy, players_id)

,White,Black,White_ID,Black_ID
0,forlat,Geforce,222267710200361995835915597876504557392,158946205730077771476317864809691404687


## df_games

In [18]:
df_games = pd.DataFrame(games)
df_games['Game_ID'] = df_games['Game_ID'].apply(id_generator)
##TODO date to datetime
df_games

,Game_ID,Date,White,White_Elo,Black,Black_Elo,ECO,Result
0,324922391373770765506623549882572670527,2021.01.31,forlat,1970,Geforce,2204,A01,0-1
1,11260618971674909162666684775239996115,2021.01.31,Geforce,2201,forlat,1973,B00,1-0
2,70760002573002850295560655531642050351,2021.01.31,forlat,1976,Geforce,2198,A20,0-1
3,112972619313039796118785083289574522990,2021.01.31,Geforce,2211,forlat,1963,B10,0-1
4,295085501773107150675334918259473716631,2021.01.31,forlat,1958,Geforce,2216,D30,1/2-1/2
5,220084237416550592495112028917383880651,2021.01.31,exeComp,2717,Ruvarashe,1976,B92,1-0
6,65998993227847818682351874244973970230,2021.01.31,Geforce,2213,forlat,1961,B00,1-0
7,173523390564831162973762856196512614401,2021.01.31,forlat,1964,Geforce,2210,C28,0-1
8,191465028604905029806043282030687474138,2021.01.31,Ruvarashe,1976,exeComp,2717,C11,0-1
9,86643213418660478279281104913835075795,2021.01.31,Geforce,2207,forlat,1967,B10,1-0


In [19]:
timestamp = game.headers["Time"]

In [20]:
timestamp[:]

'13:55:00'

## df_moves

In [21]:
df_moves = pd.DataFrame(moves_log_dict)
#TODO moves and cpl as features when loading to DB
df_moves['Game_ID'] = df_games['Game_ID']
df_moves.head(20)

,Game_ID,White,Black,ECO,FEN_moves,WhiteIsComp,Result
0,324922391373770765506623549882572670527,forlat,Geforce,A01,[rnbqkbnr/pppppppp/8/8/8/1P6/P1PPPPPP/RNBQKBNR...,Yes,0-1
1,11260618971674909162666684775239996115,Geforce,forlat,B00,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,1-0
2,70760002573002850295560655531642050351,forlat,Geforce,A20,[rnbqkbnr/pppppppp/8/8/2P5/8/PP1PPPPP/RNBQKBNR...,Yes,0-1
3,112972619313039796118785083289574522990,Geforce,forlat,B10,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,0-1
4,295085501773107150675334918259473716631,forlat,Geforce,D30,[rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR...,Yes,1/2-1/2
5,220084237416550592495112028917383880651,exeComp,Ruvarashe,B92,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,Yes,1-0
6,65998993227847818682351874244973970230,Geforce,forlat,B00,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,1-0
7,173523390564831162973762856196512614401,forlat,Geforce,C28,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,Yes,0-1
8,191465028604905029806043282030687474138,Ruvarashe,exeComp,C11,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,0-1
9,86643213418660478279281104913835075795,Geforce,forlat,B10,[rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR...,No,1-0


# search

In [22]:
def search_df(df, column, value):
    try:
        index = np.where(df[column] == value)
        df = df.iloc[index]
        if len(df) == 0:
            return f'No games found for {value}.'
        return df
    except KeyError as e:
        print(e, f'not found.')